In [ ]:
import pandas as pd
import dask.dataframe as dd
import hvplot.dask

#import sys
#print(sys.path)
df = dd.read_parquet('data/sleep_2018-21.parquet', engine='pyarrow', parse_dates=['Sleep Start Local', 'Sleep End Local'])
df.head()

In [ ]:
from bokeh.models.formatters import DatetimeTickFormatter

#formatter = DatetimeTickFormatter(days = ['%m/%d', '%a%d'])
dtf = DatetimeTickFormatter( days = '%m-%Y', months = '%m-%Y' , years = '%m-%Y')
#df.hvplot(xformatter=formatter, datashade=True)
df.hvplot(xformatter=dtf, datashade=True)

In [ ]:

#Chart: 'Deep Sleep Seconds' 'Light Sleep Seconds' 'Awake Seconds'
sleep_time = df[['Sleep Start Local', 'Sleep End Local']]
sleep_time.set_index('Sleep Start Local')
sleep_time[['Deep Sleep Minutes', 'Light Sleep Minutes','Awake Minutes']] = df[['Deep Sleep Seconds', 'Light Sleep Seconds','Awake Seconds']].div(60, axis='columns')
#sleep_time.head().violin(by='index.month')
#sleep_time.hvplot.violin(by='index.month', x='Sleep Start Local', y=['Deep Sleep Minutes', 'Light Sleep Minutes', 'Awake Minutes'])
sleep_time.hvplot.violin(by='index.day')

In [ ]:
sleep_time[['Deep Sleep Hours', 'Light Sleep Hours']] = sleep_time[['Deep Sleep Minutes', 'Light Sleep Minutes']].div(60, axis='columns')

avg_hours = sleep_time[['Sleep Start Local', 'Deep Sleep Hours', 'Light Sleep Hours']]
avg_hours.set_index('Sleep Start Local', sorted=True)

avg_hours.mean(axis='columns')
avg_hours['Sleep Hours Total'] = sleep_time['Sleep End Local'] - sleep_time['Sleep Start Local']

avg_hours['Sleep Hours Total'].astype('timedelta64[h]').hvplot.hist(xlabel='Hours', ylabel='Count', title="Sleep Hours Total")

In [ ]:
avg_hours.hvplot.violin(by='index.day')

In [ ]:
df

In [ ]:
import numpy as np

# TODO: Map this to sleep cycle data
# Sinusoidal curve fit regression
# https://scipy-lectures.org/intro/scipy/auto_examples/plot_curve_fit.html
# Seed the random number generator for reproducibility
np.random.seed(0)

x_data = np.linspace(0, 12, num=100)
y_data = 2 * np.sin(1.5 * x_data) + np.random.normal(size=100)

# And plot it
import matplotlib.pyplot as plt
plt.figure(figsize=(6, 4))
plt.scatter(x_data, y_data)

In [ ]:
from scipy import optimize

def test_func(x, a, b):
    return a * np.sin(b * x)

params, params_covariance = optimize.curve_fit(test_func, x_data, y_data,
                                               p0=[2, 1.5])

print(params)

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(x_data, y_data, label='Data')
plt.plot(x_data, test_func(x_data, params[0], params[1]),
         label='Fitted function')

plt.legend(loc='best')

plt.show()

In [ ]:
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Legend, PrintfTickFormatter
from bokeh.models.tickers import FixedTicker

# Testing sleep chart from baby tracker thing
# TODO: Figure this code out later
colors = ['#2D8E65', '#8E2D56']
p1 = figure(title="Sleep Chart", plot_width=450, plot_height=165, sizing_mode='scale_width',
           x_axis_label='Day', y_axis_label='Time of Day',
           x_range=(2.75, 18), y_range=(24,0))
sleep = p1.rect(sleepx, sleepy, 1/30, .25, color=colors[0], width_units="data", height_units="data",
        muted_color=colors[0], muted_alpha=0.2, line_alpha=0)
wake = p1.rect(wakex, wakey, 1/30, .25, color=colors[1], width_units="data", height_units="data",
        muted_color=colors[1], muted_alpha=0.2, line_alpha=0)
legend = Legend(items=[("Sleep", [sleep]), ("Wake", [wake])], orientation="horizontal", click_policy="mute")
p1.add_layout(legend, 'below')
p1.yaxis[0].formatter = PrintfTickFormatter(format="%2f:00")
p1.yaxis.ticker = FixedTicker(ticks=[0, 6, 12, 18, 24])
show(p1)